<h1>Overview</h1>

Business problem goes here.

In [1]:
# Import necessary modules
import pandas as pd
from datetime import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
import warnings

%matplotlib inline
plt.style.use('seaborn')
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.3f}'.format

I dislike using statsmodels, so I created a helper function to extract all
of the relevant information from an SKLearn regression.

In [2]:
# The following function returns the results of a sklearn model
def model_summary(model,train_X,test_X,train_y,test_y):
    #Evaluates the model on training data
    train_r2 = model.score(train_X,train_y)
    train_mae = mean_absolute_error(train_y,model.predict(train_X))
    train_mse = mean_squared_error(train_y,model.predict(train_X))
    train_rmse = mean_squared_error(train_y,model.predict(train_X),squared=False)

    #Evaluates the model on test data
    test_r2 = model.score(test_X,test_y)
    test_mae = mean_absolute_error(test_y,model.predict(test_X))
    test_mse = mean_squared_error(test_y,model.predict(test_X))
    test_rmse = mean_squared_error(test_y,model.predict(test_X),squared=False)

    labels = ['Train R2','Train Mean Abs Err','Train Mean Sq Err','Train Root Mean Sq Err',
            'Test R2','Test Mean Abs Err','Test Mean Sq Err','Test Root Mean Sq Err']
    results = [train_r2,train_mae,train_mse,train_rmse,
            test_r2,test_mae,test_mse,test_rmse]
    
    #Return the results as pandas dataframes
    dfr = pd.DataFrame(results,index=labels,columns=['Values'])
    coefficients = pd.DataFrame(model.coef_,index=train_X.columns,columns=['Values'])
    return dfr,coefficients

<h1>Datasets</h1>

The data provided 'kc_house_data.csv' contains information on thousands of homes sold in
the King County WA area. The zipcode data 'zips.csv' was used to map each house's zipcode
to the city it resides in.

In [3]:
# Homes contains data on home sales, zipcodes is for encoding cities later
homes = pd.read_csv('data/kc_house_data.csv')
zipcodes = pd.read_csv('data/zips.csv')

# Columns suggested to drop by project description
cols_to_drop = (['id','date','sqft_above','sqft_basement',
                'lat','long','sqft_living15','sqft_lot15'])
homes = homes.drop(cols_to_drop,axis=1)

# Exchange the zipcode column for a city column based on zipcode
homes = homes.merge(zipcodes,how='left',on='zipcode').drop('zipcode',axis=1)

# Overview of the data
homes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 0 to 21596
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         21597 non-null  float64
 1   bedrooms      21597 non-null  int64  
 2   bathrooms     21597 non-null  float64
 3   sqft_living   21597 non-null  int64  
 4   sqft_lot      21597 non-null  int64  
 5   floors        21597 non-null  float64
 6   waterfront    19221 non-null  object 
 7   view          21534 non-null  object 
 8   condition     21597 non-null  object 
 9   grade         21597 non-null  object 
 10  yr_built      21597 non-null  int64  
 11  yr_renovated  17755 non-null  float64
 12  city          21597 non-null  object 
dtypes: float64(4), int64(4), object(5)
memory usage: 2.3+ MB


In [4]:
# Quick statistics on the data
homes.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,yr_built,yr_renovated
count,21597.000,21597.000,21597.000,21597.000,21597.000,21597.000,21597.000,17755.000
mean,540296.574,3.373,2.116,2080.322,15099.409,1.494,1971.000,83.637
std,367368.140,0.926,0.769,918.106,41412.637,0.540,29.375,399.946
min,78000.000,1.000,0.500,370.000,520.000,1.000,1900.000,0.000
25%,322000.000,3.000,1.750,1430.000,5040.000,1.000,1951.000,0.000
50%,450000.000,3.000,2.250,1910.000,7618.000,1.500,1975.000,0.000
75%,645000.000,4.000,2.500,2550.000,10685.000,2.000,1997.000,0.000
max,7700000.000,33.000,8.000,13540.000,1651359.000,3.500,2015.000,2015.000


In [5]:
# Quick overview of the data
homes.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,yr_renovated,city
0,221900.000,3,1.000,1180,5650,1.000,NaN,NONE,Average,7 Average,1955,0.000,Seattle
1,538000.000,3,2.250,2570,7242,2.000,NO,NONE,Average,7 Average,1951,1991.000,Seattle
2,180000.000,2,1.000,770,10000,1.000,NO,NONE,Average,6 Low Average,1933,nan,Kenmore
3,604000.000,4,3.000,1960,5000,1.000,NO,NONE,Very Good,7 Average,1965,0.000,Seattle
4,510000.000,3,2.000,1680,8080,1.000,NO,NONE,Average,8 Good,1987,0.000,Sammamish


<h1>Data Cleanup & Pre-processing</h1>

Is there really a house with 33 bedrooms? Let's investigate this further.

In [6]:
homes[homes.bedrooms > 8]

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,yr_built,yr_renovated,city
4092,599999.000,9,4.500,3830,6988,2.500,NO,NONE,Average,7 Average,1938,0.000,Seattle
4231,700000.000,9,3.000,3680,4400,2.000,NO,NONE,Average,7 Average,1908,0.000,Seattle
6073,1280000.000,9,4.500,3650,5000,2.000,NO,NONE,Average,8 Good,1915,2010.000,Seattle
8537,450000.000,9,7.500,4050,6504,2.000,NO,NONE,Average,7 Average,1996,0.000,Seattle
8748,520000.000,11,3.000,3000,4960,2.000,NO,NONE,Average,7 Average,1918,1999.000,Seattle
13301,1150000.000,10,5.250,4590,10920,1.000,NO,AVERAGE,Average,9 Better,2008,0.000,Bellevue
15147,650000.000,10,2.000,3610,11914,2.000,NO,NONE,Good,7 Average,1958,0.000,Bellevue
15856,640000.000,33,1.750,1620,6000,1.000,NO,NONE,Very Good,7 Average,1947,0.000,Seattle
16830,1400000.000,9,4.000,4620,5508,2.500,NO,NONE,Average,11 Excellent,1915,0.000,Seattle
18428,934000.000,9,3.000,2820,4480,2.000,NO,NONE,Average,7 Average,1918,0.000,Seattle


The 33 bedroom house only has 1600 sq. ft. of living space. I will assume this is a 
data entry issue and impute the median value of 3 bedrooms on this home.

In [7]:
homes.loc[homes.bedrooms > 20,'bedrooms'] = 3

Waterfronts seems to have some missing values. I will impute the mode "NO" for the missing values,
and then map the values No to 0 and Yes to 1 so this feature can be used in my model.

In [8]:
waterfront_rule = {'NO':0,'YES':1}
waterfronts = homes.waterfront.fillna('NO').map(waterfront_rule)
homes.waterfront = waterfronts

Views also seems to have some missing values. I will impute the mode "NONE" for the missing values.
In order to get this column to work with my model, I will take the values from worst to best and
map them to the numbers 0 through 4.

In [9]:
view_rule = {'NONE':0,'FAIR':1,'AVERAGE':2,'GOOD':3,'EXCELLENT':4}
views = homes.view.fillna('NONE').map(view_rule)
homes.view = views

Condition is very similar to view, but without any missing values to impute. In order to get this column to work with my model, I will take the values from worst to best and map them to the numbers 0 through 4.

In [10]:
condition_rule = {'Poor':0,'Fair':1,'Average':2,'Good':3,'Very Good':4}
conditions = homes.condition.map(condition_rule)
homes.condition = conditions

Grade already has a numerical rating in the column, so I will extract that number for use in my model.

In [11]:
# Numerical rating is the part of grade before the space
grades = homes.grade.apply(lambda x: int(x.split()[0]))
homes.grade = grades

I decided to engineer a feature called since_reno which is the number of years since the last renovation.
If the home had an NA value for year renovated, I assumed the home had not been renovated and used the 
year built as the renovation date. I used a helper function since_reno to calculate this feature.

In [12]:
homes.yr_renovated.fillna(0,inplace=True)

def since_reno(home):
    # Use the year built if the house has not been renovated
    if home.yr_renovated < home.yr_built:
        return dt.today().year - home.yr_built
    else:
        return dt.today().year - home.yr_renovated
        
homes['since_reno'] = homes.apply(lambda x:since_reno(x),axis=1)
homes.drop('yr_renovated',inplace=True,axis=1)

I decided to drop the year built in favor of the home's age. This probably won't affect the model but is easier to read.

In [13]:
homes['age'] = dt.today().year - homes['yr_built']
homes.drop('yr_built',inplace=True,axis=1)

City is a categorical feature, so I had to one-hot-encode it so it would work with my model.
I did not merge cities back into my main dataset yet so the visualizations will be easier to see.

In [14]:
cities = pd.get_dummies(homes.city,prefix='city',sparse=False,drop_first=True)
# Grouped cities is created for plotting purposes later
grouped_cities = homes.copy().groupby('city')
homes.drop('city',inplace=True,axis=1)
cities.head()

,city_Bellevue,city_Black_Diamond,city_Bothell,city_Carnation,city_Duvall,city_Enumclaw,city_Fall_City,city_Federal_Way,city_Issaquah,city_Kenmore,...,city_Medina,city_Mercer_Island,city_North_Bend,city_Redmond,city_Renton,city_Sammamish,city_Seattle,city_Snoqualmie,city_Vashon,city_Woodinville
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


<h1>Exploratory Data Analysis</h1>

<h3>Distrobution of all variables</h3>

In [15]:
'''
pd.plotting.hist_frame(homes,figsize=(16,12));
'''

'\npd.plotting.hist_frame(homes,figsize=(16,12));\n'

<h3>Correlation between all variables</h3>

In [16]:
'''
fig,ax = plt.subplots(figsize = (16,12))
ax.set_title('Correlation Coefficients',fontsize=30);
sns.heatmap(homes.corr(),annot=True,ax=ax,cmap='Reds');
'''

"\nfig,ax = plt.subplots(figsize = (16,12))\nax.set_title('Correlation Coefficients',fontsize=30);\nsns.heatmap(homes.corr(),annot=True,ax=ax,cmap='Reds');\n"

<h3>Regression plot for each variable</h3>

In [17]:
'''
fig,ax = plt.subplots(3,4,figsize=(16,12))
for idx,row in enumerate(ax):
    for idx2,col in enumerate(row):
        y_val = homes.columns.values[(4*idx) + idx2]

        # Make a histogram for the price column
        if y_val == 'price':
            sns.histplot(homes.price,ax=col)
            col.set_title('Price (Millions $)')
            col.set_xticklabels(col.get_xticks()/1000000)
            col.set_xlabel('')
        # Make a scatter plot for all of the other columns
        else:
            sns.regplot(y='price',x=y_val,data=homes,ax=col,line_kws={"color": "red"})
            col.set_title(y_val.title())
            col.set_yticklabels(col.get_yticks()/1000000)
            col.set_xlabel('')
            col.set_ylabel('Price (Millions $)')
'''

'\nfig,ax = plt.subplots(3,4,figsize=(16,12))\nfor idx,row in enumerate(ax):\n    for idx2,col in enumerate(row):\n        y_val = homes.columns.values[(4*idx) + idx2]\n\n        # Make a histogram for the price column\n        if y_val == \'price\':\n            sns.histplot(homes.price,ax=col)\n            col.set_title(\'Price (Millions $)\')\n            col.set_xticklabels(col.get_xticks()/1000000)\n            col.set_xlabel(\'\')\n        # Make a scatter plot for all of the other columns\n        else:\n            sns.regplot(y=\'price\',x=y_val,data=homes,ax=col,line_kws={"color": "red"})\n            col.set_title(y_val.title())\n            col.set_yticklabels(col.get_yticks()/1000000)\n            col.set_xlabel(\'\')\n            col.set_ylabel(\'Price (Millions $)\')\n'

<h3>Median price by location</h3>

In [18]:
'''
# Reshaping the data and taking only the first 10 cities for visibility
grouped_cities = pd.DataFrame(grouped_cities['price'].agg(np.median).sort_values(ascending=False)[:10]).T
fig,ax = plt.subplots(figsize = (16,12))
sns.set_theme(context='talk');
sns.barplot(data = grouped_cities)
ax.set_xticklabels(grouped_cities.columns.values[:10], rotation = 45);
ax.set_yticklabels(ax.get_yticks()/1000000);
ax.set_title('Median Home Price by Location');
ax.set_ylabel('Price (Millions $)');
ax.set_xlabel('');
'''

"\n# Reshaping the data and taking only the first 10 cities for visibility\ngrouped_cities = pd.DataFrame(grouped_cities['price'].agg(np.median).sort_values(ascending=False)[:10]).T\nfig,ax = plt.subplots(figsize = (16,12))\nsns.set_theme(context='talk');\nsns.barplot(data = grouped_cities)\nax.set_xticklabels(grouped_cities.columns.values[:10], rotation = 45);\nax.set_yticklabels(ax.get_yticks()/1000000);\nax.set_title('Median Home Price by Location');\nax.set_ylabel('Price (Millions $)');\nax.set_xlabel('');\n"

<h1>Models</h1>

In [19]:
# Plotting is done so we can bring the encoded cities data back
homes = pd.concat([homes,cities],axis=1)

<h3>Basic Model</h3>

This is a very basic model that only used sqft_living to predict the price

In [20]:
X_basic = homes[['sqft_living']]
y_basic = homes.price

# Perform a train test split with the default size
X_train_basic, X_test_basic, y_train_basic, y_test_basic\
     = train_test_split(X_basic,y_basic,random_state=271828)

# Create and fit linear regression and get summary
basic_regression = LinearRegression().fit(X_train_basic,y_train_basic)
basic_summary = model_summary(basic_regression,
                              X_train_basic,X_test_basic,y_train_basic,y_test_basic)
# Print summary statistics
basic_summary[0]

,Values
Train R2,0.488
Train Mean Abs Err,175043.731
Train Mean Sq Err,70579154795.151
Train Root Mean Sq Err,265667.376
Test R2,0.509
Test Mean Abs Err,171258.475
Test Mean Sq Err,62125867519.457
Test Root Mean Sq Err,249250.612


In [23]:
# Print coefficients
basic_summary[1]

,Values
sqft_living,282.439


An R2 of around 0.5 for both train and test is not great and the RMSE is around $250,000. This means the model is off by a quarter of a million dollars on average. Not good!